In [39]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
settings = {
    'Body': ['Display > PER_PRO_NF_Body_Innkeepr_HighImpact_Seed1',
       'Display > PER_PRO_NF_Body_Innkeepr_HighImpact_Seed2',
       'Display > PER_PRO_NF_Body_Innkeepr_LikelyPurchaser_Seed1',
       'Display > PER_PRO_NF_Body_Innkeepr_LikelyPurchaser_Seed2',
       'Display > Retargeting_Body'],
    'Kids': [
       'Display > PER_PRO_NF_Kids_Innkeepr_HighImpact_Seed1',
       'Display > PER_PRO_NF_Kids_Innkeepr_HighImpact_Seed2',
       'Display > PER_PRO_NF_Kids_Innkeepr_LikelyPurchaser_Seed1',
       'Display > PER_PRO_NF_Kids_Innkeepr_LikelyPurchaser_Seed2',
       'Display > Retargeting_Kids'
    ]
}
rolling_weeks = 3

In [41]:
files = glob.glob("/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/Evaluations/Tchibo-Criteo/Reports/*.csv")
data_list = []
df = pd.DataFrame()
for file in files: 
    print(f"Read file {file}")
    if "Innkeepr_Criteo_Jan2026" in file:
        skip_rows = 6
    else:
        skip_rows = 7
    temp = pd.read_csv(file, sep=";", skiprows=skip_rows)
    print(temp["Line Item"].value_counts())
    data_list.append(temp)
    print(temp.shape, len(data_list))
df = pd.concat(data_list)


In [42]:
use_columns = [col for col in df.columns if "Unnamed" not in col]
df = df[use_columns]
df = df.reset_index(drop=True)
cost_column = "Costs (Total)"
df = df.dropna(subset=["Line Item"])
df = df[df["Week"]!="2026-03"]
df[cost_column] = df[cost_column].str.replace('.', '').str.replace(',', '.').astype(float)
df["CDR"] = df["Tchibo CDR Net"].str.replace('%', '').str.replace('.', '').str.replace(',', '.').astype(float)
df[f"rolling_{cost_column}"] = df.groupby("Line Item")[cost_column].rolling(window=rolling_weeks).mean().reset_index(0, drop=True)
df["rolling_CDR"] = df.groupby("Line Item")["CDR"].rolling(window=rolling_weeks).mean().reset_index(0, drop=True)
df = df.sort_values(by=["Line Item", "Week"], ascending=True).reset_index(drop=True)
df[["Line Item", "Week", "Tchibo CDR Net", cost_column, "CDR", f"rolling_{cost_column}", "rolling_CDR"]]

In [43]:
campaings = df["Line Item"].unique()
campaings

In [44]:
key = "Body"
temp = df[df["Line Item"].isin(settings[key])]
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(211)
ax.set_title("Costs")
sns.lineplot(data=temp, x="Week", y=cost_column, hue="Line Item", marker="o", ax=ax)
plt.xticks(rotation=90)
plt.tight_layout()
plt.grid(True)
ax2 = fig.add_subplot(212)
ax2.set_title("Tchibo CDR Net")
sns.lineplot(data=temp, x="Week", y="CDR", hue="Line Item", marker="o", ax=ax2)
plt.xticks(rotation=90)
plt.ylim(0, 40)
plt.tight_layout()
plt.grid(True)
plt.show()


In [49]:
key = "Body"
temp = df[df["Line Item"].isin(settings[key])]
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(211)
ax.set_title(f"Rolling Mean Costs ({rolling_weeks} weeks)")
sns.lineplot(data=temp, x="Week", y=f"rolling_{cost_column}", hue="Line Item", marker="o", ax=ax)
plt.xticks(rotation=90)
plt.tight_layout()
plt.grid(True)
ax2 = fig.add_subplot(212)
ax2.set_title(f"Rolling Tchibo CDR Net  Costs ({rolling_weeks} weeks)")
sns.lineplot(data=temp, x="Week", y="rolling_CDR", hue="Line Item", marker="o", ax=ax2)
plt.xticks(rotation=90)
plt.ylim(0, 40)
plt.tight_layout()
plt.grid(True)
plt.show()

In [46]:
key = "Kids"
temp = df[df["Line Item"].isin(settings[key])]
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(211)
ax.set_title("Costs")
sns.lineplot(data=temp, x="Week", y=cost_column, hue="Line Item", marker="o", ax=ax)
plt.xticks(rotation=90)

plt.tight_layout()
plt.grid(True)
ax2 = fig.add_subplot(212)
ax2.set_title("Tchibo CDR Net")
sns.lineplot(data=temp, x="Week", y="CDR", hue="Line Item", marker="o", ax=ax2)
plt.xticks(rotation=90)
plt.ylim(0, 40)
plt.tight_layout()
plt.grid(True)
plt.show()


In [50]:
key = "Kids"
temp = df[df["Line Item"].isin(settings[key])]
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(211)
ax.set_title(f"Rolling Mean Costs ({rolling_weeks} weeks)")
sns.lineplot(data=temp, x="Week", y=f"rolling_{cost_column}", hue="Line Item", marker="o", ax=ax)
plt.xticks(rotation=90)
plt.tight_layout()
plt.grid(True)
# ax2 = fig.add_subplot(312)
# ax2.set_title("Tchibo CDR Net Costs")
# sns.lineplot(data=temp, x="Week", y="CDR", hue="Line Item", marker="o", ax=ax2)
# ax2.set_ylabel("Tchibo CRD Net")
# plt.grid(True)
# plt.xticks(rotation=90)
# plt.tight_layout()
ax3 = fig.add_subplot(212)
ax3.set_title(f"Rolling Tchibo CDR Net  Costs ({rolling_weeks} weeks)")
sns.lineplot(data=temp, x="Week", y="rolling_CDR", hue="Line Item", marker="o", ax=ax3)
plt.xticks(rotation=90)
plt.ylim(0, 40)
plt.tight_layout()
plt.grid(True)
plt.show()